<a href="https://colab.research.google.com/github/wisesprout/newspaper_to_online_newsletter/blob/main/Nextlab_news_category_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-n0vchsgb
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-n0vchsgb
     |████████████████████████████████| 132 kB 8.5 MB/s 
     |████████████████████████████████| 344 kB 59.7 MB/s 
     |████████████████████████████████| 47.3 MB 1.2 MB/s 
     |████████████████████████████████| 4.5 MB 43.1 MB/s 
     |████████████████████████████████| 1.2 MB 49.4 MB/s 
     |████████████████████████████████| 3.8 MB 57.3 MB/s 
     |████████████████████████████████| 596 kB 56.1 MB/s 
     |████████████████████████████████| 6.5 MB 55.1 MB/s 
     |████████████████████████████████| 895 kB 49.4 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
     |████████████████████████████████| 79 kB 8.4 MB/s 
     |████████████████████████████████| 8.6 MB 26.1 MB/s 
     |████████████████████████████████| 127 kB 57.2 MB/s 
  Created wheel for kobert: filename=kob

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, notebook

import os

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
# 사용할 device 설정
device = torch.device('cuda')  # GPU에 저장하고 GPU에서 불러오기

# get pretrained model, bert and vocab
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


#Load Data

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- news_class9x1400 = 9개 카테고리별 1400개 기사
- 1개 기사 = 줄 바꿈으로 대부분의 문장 구분


In [5]:
dataset_train = []
dataset_test = []

root = "/content/drive/MyDrive/Colab Notebooks/CP2/news_class9x1400/"
list = os.listdir(root)

# delete '.DS_Store'
for cat in list:
    if cat == '.DS_Store':
        list.remove(cat)

# create dictionary for news categories
cat_dict = {}
for cat in list:
    cat_dict[list.index(cat)] = cat
print('번호에 해당하는 카테고리:', cat_dict)

for cat in list:   # 파일을 읽어서 기사 내용 + 카테고리를 하나의 list 데이터로 생성
    files = os.listdir(root + cat)  # n번째 카테고리의 파일 경로
    for i,f in enumerate(files):  # i=index, f=i번째 기사 파일명
        fname = root + cat + "/" + f  # i번째 기사 파일 경로
        file = open(fname, "r")  # 'r'=read
        strings = file.read()

        cat_num = str(list.index(cat))  # 카테고리의 인덱스

        # train:test = 17:3 = 1190개:210개
        if i < 1190:
            dataset_train.append([strings, cat_num])  # 카테고리 이름 대신 인덱스를 str으로 대입
        else:
            dataset_test.append([strings, cat_num])  # 카테고리 이름 대신 인덱스 str으로 대입
        file.close()

# train set = 1190개 기사 * 9개 카테고리 = 10710, 
# test set = 210개 기사 * 9개 카테고리 = 1890
print(len(dataset_train), len(dataset_test))  

번호에 해당하는 카테고리: {0: 'culture', 1: 'politic', 2: 'social', 3: 'life', 4: 'health', 5: 'sport', 6: 'economy', 7: 'entertainment', 8: 'ITscience'}
10710 1890


In [6]:
# first data of train set
# 뉴스데이터, 카테고리 번호
dataset_train[0]

['자꾸 읽다가 정들어버린 〈탈북 여대생〉\n책을 읽는 일, 책을 만드는 일. 오랫동안 꿈꿔왔던 일을 하기 위해 출판사에 발을 내딛었습니다.\n그리고 첫 출근날, 처음으로 맞이했던 원고는 교정이 거의 완성된 원고였기에 심하게(?) 교정교열을 볼 필요는 없었지만 사소한 맞춤법, 띄어쓰기 등을 교정하였지요.\n의문나는 점은 저자와 상의하면서 원고를 다듬어 나갔고, 원고를 다 살펴보고 나서는 예쁘게 포장해 줄 카피를 연구하고 보도 자료를 작성했습니다.\n근데 참 이상하지요, 그런 상황이 계속될수록 처음엔 완전 최고다라는 느낌과는 거리가 멀었던 원고의 매력이 점점 느껴졌어요.\n여러 번 읽다 보니 결국! 원고를 좋아하게 되었습니다.\n마침내 책으로 나왔을 때는 제법 괜찮은 책이라는 생각마저 들더라구요.\n무슨 책이냐구요?\n그게 바로 <탈북 여대생>입니다.\n<탈북 여대생>은 대하소설 <고구려-전7권>를 발간한 작가 정수인의 소설집입니다.\n표제작인 탈북 여대생과 여우 두 작품이 수록되어 있어요.\n처음 원고로 접했을 때는 한 권의 소설집에 함께 수록되는 것이 어색하게 느껴질 정도로 두 작품이 너무 다르게 느껴졌어요.\n우선 탈북 여대생 내용을 살펴볼까요?\n화자인 소설가 나는 연변에 살고 있습니다.\n북한에 관한 소설을 쓰기 위해 탈북자들과 조선족들을 만나면서 북조선 새기(아가씨, 처녀를 일컫는 말) 설화를 알게 됩니다.\n얼굴이 곱디고운 설화는 남자친구 강철과 헤어지고 강철이 없는 곳으로 돌아가고 싶지 않아 두만강을 건넜다고 합니다.\n자존심이 강해서 북한의 가난한 현실에 대해서 제대로 말하려고 하지 않는 설화. 그런 설화에게 나는 다그칩니다.\n배고픈 사람이 식량을 달라고 해야지 아무 소리도 하지 않고 있으면 누가 그냥 준다니? 너 같은 사람이 자존심 때문에 입을 다물고 있으면 북조선 인민들은 그만큼 고통이 길어지는 거야.\n다 알고 있는 일이라도 자꾸 이야기해야 돼.\n신문이나 방송에서도 북조선 인민들이 굶고 있다고 보도를 하고 있지만 더 많은 사람들이 자꾸 

In [7]:
# tokenize vocab
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [8]:
# dataset class for training
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, 
                                                   max_seq_length=max_len, 
                                                   pad=pad, 
                                                   pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [9]:
# setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [10]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

# DataLoader = samples instances(document+label) from the dataset to batches
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#Make Model

In [11]:
# model for classification
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=9,  # for 9 categories
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out) 

In [12]:
# necessities for training
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [13]:
# accuracy while training
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

#Train Model

In [14]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/168 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 train acc 0.4471979843073593


  0%|          | 0/30 [00:00<?, ?it/s]

epoch 1 test acc 0.7550551470588236


...

epoch 9 test acc 0.791329656862745


  0%|          | 0/168 [00:00<?, ?it/s]

epoch 10 train acc 0.9921959550865802


  0%|          | 0/30 [00:00<?, ?it/s]

epoch 10 test acc 0.7856924019607843


In [15]:
'''# saving model in Google Drive
model_name = 'cat_classifier.pt'
path = f'/content/drive/MyDrive/Colab Notebooks/CP2/{model_name}'
torch.save(model.state_dict(), path)

# how to load model from Google Drive
model.load_state_dict(torch.load(path))'''

"# saving model in Google Drive\nmodel_name = 'cat_classifier.pt'\npath = f'/content/drive/MyDrive/Colab Notebooks/CP2/{model_name}'\ntorch.save(model.state_dict(), path)\n\n# how to load model from Google Drive\nmodel.load_state_dict(torch.load(path))"

#Test

In [17]:
# function for testing any input text
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/CP2/news.pt")
modelload = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
modelload.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/CP2/news.pt", device))

def testModel(model, seq):
    # 'culture', 'politic', 'social', 'life', 'health', 'sport', 'economy', 'entertainment', 'ITscience'
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    modelload.eval()
    # torch.tensor = turns python list to pytorch tensor
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    #idx = result.argmax().cpu().item()
    idx = result.argmax().cuda().item()
    print("뉴스의 카테고리는:", cat_dict[idx])
    print("신뢰도는:", "{:.2f}%".format(softmax(result,idx)))

In [18]:
# testing
testModel(model, "신형 아이패드 프로에 m1칩 탑재 예정")

뉴스의 카테고리는: ITscience
신뢰도는: 99.75%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
